In [1]:
import coremltools
import coremltools.proto.FeatureTypes_pb2 as ft

In [2]:
spec = coremltools.utils.load_spec("./Original CoreML Models/test01_model.mlmodel")

In [3]:
spec_layers = getattr(spec,spec.WhichOneof("Type")).layers

In [4]:
# find the current output layer and save it for later reference
last_layer = spec_layers[-1]

In [5]:
# add the post-processing layer
new_layer = spec_layers.add()
new_layer.name = 'convert_to_image'

In [6]:
# Configure it as an activation layer
new_layer.activation.linear.alpha = 255
new_layer.activation.linear.beta = 0

In [7]:
# Use the original model's output as input to this layer
new_layer.input.append(last_layer.output[0])

In [8]:
# Name the output for later reference when saving the model
new_layer.output.append('image_output')

In [9]:
# Find the original model's output description
output_description = next(x for x in spec.description.output if x.name==last_layer.output[0])
 
# Update it to use the new layer as output
output_description.name = new_layer.name

In [10]:
output = spec.description.output[0]
output

name: "convert_to_image"
shortDescription: "Compressed Image Output"
type {
  multiArrayType {
    shape: 1
    shape: 3
    shape: 512
    shape: 768
    dataType: FLOAT32
  }
}

In [11]:
del output.type.multiArrayType.shape[0]

In [12]:
# Function to mark the layer as output
# https://forums.developer.apple.com/thread/81571#241998
def convert_multiarray_output_to_image(spec, feature_name, is_bgr=False): 
    """ 
    Convert an output multiarray to be represented as an image 
    This will modify the Model_pb spec passed in. 
    Example: 
        model = coremltools.models.MLModel('MyNeuralNetwork.mlmodel') 
        spec = model.get_spec() 
        convert_multiarray_output_to_image(spec,'imageOutput',is_bgr=False) 
        newModel = coremltools.models.MLModel(spec) 
        newModel.save('MyNeuralNetworkWithImageOutput.mlmodel') 
    Parameters 
    ---------- 
    spec: Model_pb 
        The specification containing the output feature to convert 
    feature_name: str 
        The name of the multiarray output feature you want to convert 
    is_bgr: boolean 
        If multiarray has 3 channels, set to True for RGB pixel order or false for BGR 
    """
    for output in spec.description.output: 
        if output.name != feature_name: 
            continue
        if output.type.WhichOneof('Type') != 'multiArrayType': 
            raise ValueError("%s is not a multiarray type" % output.name) 
        array_shape = tuple(output.type.multiArrayType.shape) 
        channels, height, width = array_shape 
        from coremltools.proto import FeatureTypes_pb2 as ft 
        if channels == 1: 
            output.type.imageType.colorSpace = ft.ImageFeatureType.ColorSpace.Value('GRAYSCALE') 
        elif channels == 3: 
            if is_bgr: 
                output.type.imageType.colorSpace = ft.ImageFeatureType.ColorSpace.Value('BGR') 
            else: 
                output.type.imageType.colorSpace = ft.ImageFeatureType.ColorSpace.Value('RGB') 
        else: 
            raise ValueError("Channel Value %d not supported for image inputs" % channels) 
        output.type.imageType.width = width 
        output.type.imageType.height = height 

In [13]:
# Mark the new layer as image
convert_multiarray_output_to_image(spec, output_description.name, is_bgr=False)

In [14]:
updated_model = coremltools.models.MLModel(spec)
 
model_file_name = './updated_model.mlmodel'
updated_model.save(model_file_name)

/Users/anujdutt/miniconda3/envs/PyTorch/lib/python3.8/site-packages/coremltools/models/model.py:115: RuntimeWarning: You will not be able to run predict() on this Core ML model. Underlying exception message was: Error compiling model: "Error reading protobuf spec. validator error: Interface specifies output 'convert_to_image' which is not produced by any layer in the neural network.".
  _warnings.warn(


In [ ]:
output = spec.description.output[0]
output

In [ ]:
del output.type.multiArrayType.shape[0]

In [ ]:
output

In [ ]:
output.type.imageType.colorSpace = ft.ImageFeatureType.RGB 
output.type.imageType.height = 768
output.type.imageType.width = 512

In [ ]:
output

In [ ]:
coremltools.utils.save_spec(spec, "./Final_Models/test01_model.mlmodel")

In [ ]:
def convert_multiarray_to_image(feature, is_bgr=False):
    import coremltools.proto.FeatureTypes_pb2 as ft

    if feature.type.WhichOneof("Type") != "multiArrayType":
        raise ValueError("%s is not a multiarray type" % feature.name)

    shape = tuple(feature.type.multiArrayType.shape)
    channels = None
    if len(shape) == 2:
        channels = 1
        height, width = shape
    elif len(shape) == 3:
        channels, height, width = shape

    if channels != 1 and channels != 3:
        raise ValueError("Shape {} not supported for image type".format(shape))

    if channels == 1:
        feature.type.imageType.colorSpace = ft.ImageFeatureType.GRAYSCALE
    elif channels == 3:
        if is_bgr:
            feature.type.imageType.colorSpace = ft.ImageFeatureType.BGR
        else:
            feature.type.imageType.colorSpace = ft.ImageFeatureType.RGB

    feature.type.imageType.width = width
    feature.type.imageType.height = height

In [ ]:
convert_multiarray_to_image(spec.description.output[0], is_bgr=False)

In [ ]:
output

In [ ]:
coremltools.utils.save_spec(spec, "./Final_Models/test01_model.mlmodel")

In [ ]:
# Test
new_spec = coremltools.utils.load_spec("./Final_Models/test01_model.mlmodel")

In [ ]:
output = new_spec.description.output[0]
output